<a href="https://colab.research.google.com/github/manmustbecool/Experiment/blob/main/llm_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets

In [ ]:
## test

# Import necessary libraries
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding
import torch


# Initialize tokenizer and model
model_name = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


# Sample dataset
sample_data = [
    {"prompt": "wer?", "response": "no"},
    {"prompt": "wer wer?", "response": "no no no"},
]*300

# Load dataset
dataset = Dataset.from_list(sample_data)

# Tokenization function with uniform padding
def preprocess_function(example):
    inputs = tokenizer(example["prompt"], truncation=True, padding="max_length", max_length=64, return_tensors="pt")
    labels = tokenizer(example["response"], truncation=True, padding="max_length", max_length=64, return_tensors="pt")["input_ids"]
    return {"input_ids": inputs["input_ids"].squeeze(0), "labels": labels.squeeze(0)}

tokenized_dataset = dataset.map(preprocess_function)

# Define data collator for consistent tensor shapes
data_collator = DataCollatorWithPadding(tokenizer)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=30,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=1,
    label_names=["labels"],
    report_to="none"
)

# Initialize Trainer with data collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator  # Handles padding dynamically
)

# Fine-tune the model
trainer.train()

model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

In [ ]:
# Initialize tokenizer and model
model_name = "./fine_tuned_model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Test the model with a sample prompt
input_text = "wer?"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(inputs["input_ids"], max_length=20) # Generate response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Model Output:", response)

In [ ]:
# Initialize tokenizer and model
model_name = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Test the model with a sample prompt
input_text = "wer?"
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(inputs["input_ids"], max_length=20) # Generate response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Model Output:", response)

In [2]:
from transformers import pipeline

generator = pipeline('text-generation', model="facebook/opt-125m")
generator("wer?")

Device set to use cpu


[{'generated_text': "wer?\nI'm not sure, but I think it's a good idea."}]